<a href="https://colab.research.google.com/github/MatteoGuglielmi-tech/Polarity-and-Subjectivity-Detection/blob/main/src/MyModel/BERT-Fine-Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT Embedding Fine Tuning

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.3 MB/s 
     |████████████████████████████████| 163 kB 43.9 MB/s 
     |████████████████████████████████| 7.6 MB 11.1 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
rootdir = '/content/gdrive/MyDrive/Colab Notebooks/Polarity-Subjectivity-Detection/'

In [4]:
import tensorflow as tf
import torch

# Get the GPU device name
device = tf.test.gpu_device_name()

if 'GPU' in device:
  print(f'GPU available : {device}')
  device = torch.device('cuda'+device[-2:])
else :
  device = torch.device("cpu")
  raise SystemError("GPU not found, use CPU instead")

SystemError: ignored

In [5]:
device

device(type='cpu')

In [49]:
import pandas as pd


# loading dataset
movie_reviews = pd.read_csv(rootdir+'Datasets/movie_rews_raw.csv')
subj_obj_dataset = pd.read_csv(rootdir+'Datasets/subj_obj_dataset_clean.csv')

In [50]:
movie_reviews

,Unnamed: 0,text
0,0,"plot : two teen couples go to a church party ,..."
1,1,the happy bastard ' s quick movie review damn ...
2,2,it is movies like these that make a jaded movi...
3,3,""" quest for camelot "" is warner bros . ' first..."
4,4,synopsis : a mentally unstable man undergoing ...
...,...,...
1995,1995,wow ! what a movie . it ' s everything a movie...
1996,1996,"richard gere can be a commanding actor , but h..."
1997,1997,"glory -- starring matthew broderick , denzel w..."
1998,1998,steven spielberg ' s second epic film on world...


In [51]:
subj_obj_dataset

,Unnamed: 0,text
0,0,"smart and alert , thirteen conversations about..."
1,1,"color , musical bounce and warm seas lapping o..."
2,2,it is not a mass-market entertainment but an u...
3,3,a light-hearted french film about the spiritua...
4,4,my wife is an actress has its moments in looki...
...,...,...
9995,9995,"in the end , they discover that balance in lif..."
9996,9996,a counterfeit 1000 tomin bank note is passed i...
9997,9997,enter the beautiful and mysterious secret agen...
9998,9998,after listening to a missionary from china spe...


In [10]:
#subjective = subj_obj_dataset[['text','labels']][:len(subj_obj_dataset)//2].sample(n=1000)

In [11]:
#objective = subj_obj_dataset[['text', 'labels']][len(subj_obj_dataset)//2:].sample(n=1000)

In [12]:
#subj_obj_dataset = pd.concat([subjective,objective], axis=0)

In [13]:
#subj_obj_dataset

In [52]:
subjective = subj_obj_dataset[['text']][:len(subj_obj_dataset)//2].sample(n=1000)
objective = subj_obj_dataset[['text']][len(subj_obj_dataset)//2:].sample(n=1000)
subj_obj_dataset = pd.concat([subjective,objective], axis=0)

In [53]:
subj_obj_dataset

,text
188,"the acting is uniformly excellent , with kyra ..."
2652,crush could be the worst film a man has made a...
76,so relentlessly wholesome it made me want to s...
2787,dong doesn't demonize the conservative christi...
3158,"oscar wilde's masterpiece , the importance of ..."
...,...
9639,he awakes to what appears to be the next morni...
9318,"a british television personality , terry twill..."
7666,he soon finds himself plunged into a mystery o...
8467,"but slowly , their teenage-son alan ( kristian..."


### Major commands :
- .tokenize(sent)
- .convert_tokens_to_ids(tokenized_sent)
- .encode.plus() [source](https://huggingface.co/docs/transformers/internal/tokenization_utils#transformers.tokenization_utils_base.PreTrainedTokenizerBase.batch_encode_plus)

In [54]:
# BERT model script from: huggingface.co
import transformers
from transformers import BertTokenizer, BertModel
from typing import Tuple, List, Dict
import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import logging
import gc

# to not see warning everytime
logging.set_verbosity_error()

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
model = BertModel.from_pretrained("bert-base-uncased").to(device)

In [55]:
def embedding(dataset : pd.DataFrame, 
              sentence_column: str, 
              tokenizer: BertTokenizer, 
              maxlen: int
              ) -> Tuple[torch.Tensor]:  
    '''Extract embedding information using passed tokenizer and model
        Params:
        ------
            dataset : pd.DataFrame
                dataframe containig sentences to be encoded
            sentence_column : str
                column in the beforementioned dataframe to be encoded
            tokenizer : transformers.models.bert.tokenization_bert.BertTokenizer
                tokenizer used to process sentences and extract related information
        Return:
        ------
            Return encodings and attention masks as pytorch tensors
    '''
    
    embeddings = {
        'embedding' : [],
        'attention_mask' : []
        }
        
    for sent in dataset[sentence_column]:
        dic_sent_encoding = tokenizer.encode_plus(sent,              # untokenized sentence
                                                add_special_tokens = True,      # add '[CLS]' and '[SEP]'
                                                truncation = True,              # truncate to maximum length
                                                max_length = maxlen,            # due to ram limiatation 
                                                padding = "max_length",         # pad to maximum admissible sentence
                                                return_attention_mask = True,   # return attention mask
                                                return_tensors = "pt")          # returns pytorch tensors
    
        # extracting embeddings and attention masks in list form
        embeddings['embedding'].append(dic_sent_encoding['input_ids'])
        embeddings['attention_mask'].append(dic_sent_encoding['attention_mask'])

    # convert lists of tensors into tensors
    #input_ids = torch.cat(embeddings['embedding'], axis=0)
    #attention_masks = torch.cat(embeddings['attention_mask'], axis=0)
    
    #return input_ids, attention_masks
    return embeddings['embedding'], embeddings['attention_mask']

In [18]:
#mr_encoding, attention_masks_mr = embedding(dataset=movie_reviews, sentence_column='text', tokenizer=tokenizer)

In [19]:
#to display a sample tensor uncomment following lines
#mr_encoding[0], attention_masks_mr[0]

In [20]:
#so_encoding, attention_masks_so = embedding(dataset=subj_obj_dataset, sentence_column='text', tokenizer=tokenizer)

In [21]:
#to display a sample tensor uncomment following lines
#so_encoding[0], attention_masks_so[0]

In [22]:
#to investigate size of a tensor uncomment following lones
#so_encoding[0].size(), attention_masks_so[0].size()

In [23]:
#BATCH_SIZE = 128 # reduced due to RAM limitations

In [56]:
def embeddings(dataset: pd.DataFrame, 
               tokenizer: BertTokenizer, 
               model: BertModel, 
               maxlen: int, 
               column_name: str='text',
               device: str=device
               )-> List:
    '''Function to batch sentences to fit BERT model and get embeddings

        Params:
        ------
            dataset : pd.DataFrame
                dataset to be batched
            column_name : str
                column of 'dataset' DataFrame to catch 
        
        Return:
        ------
            tuple of dataloader to iterate over
    '''

    model.eval()

    embs = []
    # getting encodings and attention masks for whole dataset
    emb, msk = embedding(dataset=dataset, sentence_column=column_name, tokenizer=tokenizer, maxlen=maxlen)

    with torch.no_grad():
        for e in zip(emb,msk):
            last_hidden_states = model(e[0].to(device),e[1].to(device))
            #print(last_hidden_states[0].shape)
            embs.append(last_hidden_states[0])

    return embs

In [57]:
import os, shutil

def clear_folder_content(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [26]:
clear_folder_content(rootdir+'BERT/Polarity-Embeddings')
clear_folder_content(rootdir+'BERT/Subjectivity-Embeddings')

In [27]:
clear_folder_content(rootdir+'BERT/Polarity-Embeddings-Truncated')
clear_folder_content(rootdir+'BERT/Subjectivity-Embeddings-Truncated')

In [ ]:
clear_folder_content(rootdir+'BERT/Subjectivity-Embeddings-Truncated-Raw')
clear_folder_content(rootdir+'BERT/Subjectivity-Embeddings-Truncated-Raw')

In [ ]:
def save_embs(embs: List[torch.Tensor], dest_folder: str, kind: str) -> None:
    for i in range(20):
        to_save = []
        data = embs[i*100:(i+1)*100]
        #print(f"{i}, {data}\n")
        #print("=============================================================\n")
        #print(len(data))
        for b in data: # getting one tensor at a time -> embs[i]
            #for s in b: # to have (512x768)
                #print(f"Print s {s} \n")
            to_save.append(b[0].cpu().numpy())
        np.save(rootdir+'BERT/{}/bert_emb_{}_{}.npy'.format(dest_folder, kind, (i+1)*100), to_save)

In [ ]:
mr_embs = embeddings(dataset=movie_reviews, tokenizer=tokenizer, model=model, maxlen=45)

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
subj_embs = embeddings(dataset=subj_obj_dataset, tokenizer=tokenizer, model=model, maxlen=45)

In [32]:
print(len(mr_embs))

2000


In [33]:
mr_embs[0].size()

torch.Size([1, 45, 768])

In [34]:
mr_embs[0][0].size()

torch.Size([45, 768])

In [35]:
mr_embs[0][0] == mr_embs[0]

tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]]])

In [ ]:
save_embs(embs=mr_embs, dest_folder='Polarity-Embeddings-Truncated-Raw', kind='pol')

In [ ]:
save_embs(embs=subj_embs, dest_folder='Subjectivity-Embeddings-Truncated-Raw', kind='subj')

In [ ]:
bert_emb_100 = np.load(rootdir+'BERT/Polarity-Embeddings-Truncated/bert_emb_pol_100.npy')
bert_emb_200 = np.load(rootdir+'BERT/Polarity-Embeddings-Truncated/bert_emb_pol_200.npy')

In [ ]:
bert_emb_100.shape

In [ ]:
bert_emb_200.shape

In [ ]:
bert_emb_100[:5]==bert_emb_200[:5]

In [ ]:
torch.cuda.empty_cache()
gc.collect()

### Unused

In [ ]:
#ids_dataloader, _ = batching_data(dataset=movie_reviews, tokenizer=tokenizer)
#for idx, ids in enumerate(ids_dataloader):
#    print(idx, ids, len(ids))
#    if idx==2:
#        break

In [ ]:
# embedding taken from last layer of BERT
# avoid touching and computing gradients -> torch.no_grad()
# https://towardsdatascience.com/what-is-npy-files-and-why-you-should-use-them-603373c78883

def fine_tune_BERT( model: BertModel, dataset: pd.DataFrame, tokenizer: BertTokenizer,
                   sentence_column: str='text', batch_size: int=BATCH_SIZE, 
                   device: str=device, rootdir: str=rootdir, filename: str='pol') -> List[torch.Tensor]:
    """Return Embeddings of dataset

        Params:
        ------
            model: transformers.models.bert.modeling_bert.BertModel
                BertModel to get embeddings from
            dataset : pd.Dataframe
                dataframe containing sentences to embed
            tokenizer : transformers.models.bert.tokenization_bert.BertTokenizer
                tokenizer model to use
            sentence_column : str
                column of 'dataset' to get
            batch_size : int
                batch size
            device : str
                device to load data on
            rootdir : str
                root directory where to store embeddings
            filename : str
                name on which saving embeddings
        
        Return:
        ------
            List of embeddings
    """

    embs = []
    # getting dataloaders
    ids_dataloader, msk_dataloader = batching_data(dataset=dataset, tokenizer=tokenizer, batch_size=batch_size)
    # disabling gradients computation --> I'm using a pre-trained net. Don't want to rewrite weights
    with torch.no_grad():
        # iterating through batches
        for idx, (ids,msk) in enumerate(zip(ids_dataloader,msk_dataloader)):

            # move data to device 
            ids = ids.to(device)
            msk = msk.to(device)

            # 'forward pass'
            outputs = model(ids,msk)

            # extracting tensor at last layer : https://github.com/esrel/NLU.Lab.2022.Public/blob/master/notebooks/10_sequence_nn.ipynb
            last_hidden_state = outputs.last_hidden_state

            print(f"Done batch #{idx}")
            print(last_hidden_state.cpu().numpy().shape)
            embs.append(last_hidden_state)
        if len(embs)>0:
            print('Embs list has been successfully built')
        else:
            raise ValueError('List has not been filled')

    return embs

In [ ]:
#subj_obj_embs = np.load(rootdir+'subj_obj_10000.npy')
#for i in range(1000, 11000, 1000):
#    subj_obj_embs = np.concatenate((subj_obj_embs, np.load(rootdir+f'subj_obj_{i}.npy')), axis=0)

#save_embs(dataset=subj_obj_embs, filename='subj_obj_embs.npy')

In [ ]:
#save_embs(embs=embs_per_batch_subj_obj, filename='subj_obj.npy')

In [ ]:
#subj_obj = {}

# 64*15+40 = 1000
#for i in range(1000,11000,1000):
    #subj_obj[str(i)] = subj_obj_dataset[i-1000:i]
    #embs_per_batch_subj_obj = fine_tune_BERT(model=model, dataset=subj_obj_dataset[i-1000:i], tokenizer=tokenizer)
    #save_embs(embs_per_batch_subj_obj, f'subj_obj_{i}.npy')

In [ ]:
#save_embs(embs=mr_embs, filename='pol_embs.npy')

In [ ]:
#save_embs(embs=subj_embs, filename='subj_obj_embs.npy')